# Algorithms 202: Coursework 1 Task 2: Levenshtein Distance

Group-ID: 23

Group members: 
* John Oliver (jpo14)
* Bálint Rikker (br1414)
* Peng Peng (pp3613)

# Objectives

The aim of this coursework is to enhance your algorithmic skills by mastering the divide and conquer and dynamic programming strategies. You are asked to show that you can:

- implement a dynamic programming problem

This notebook *is* the coursework. It contains cells with function definitions that you will need to complete. You will submit this notebook as your coursework.

The comparisons of different algorithms involve textual descriptions and graphical plots. For graphing you will be using [matplotlib](http://matplotlib.org/index.html) to generate plots. [This tutorial](http://matplotlib.org/index.html) will be useful to go through to get you up to speed. For the textual descriptions you may wish to use [LaTeX](http://en.wikipedia.org/wiki/LaTeX) in-line like $\mathcal{O}(n\log{}n)$. Double click this cell to reveal the required markup - and [see here](http://texblog.org/2014/06/24/big-o-and-related-notations-in-latex/) for useful guidance on producing common symbols used in asymptotic run time analysis.

# Preliminaries: helper functions

Here we define a collection of functions that will be useful for the rest of the coursework. You'll need to run this cell to get started.

In [1]:
# so our plots get drawn in the notebook
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
from sys import maxsize
from time import clock
from urllib.request import urlretrieve

# a timer - runs the provided function and reports the
# run time in ms
def time_f(f):
    before = clock()
    f()
    after = clock()
    return after - before

# we can get a word list from here - we download it once
# to 'wordlist.txt' and then reuse this file.
url = 'http://www.doc.ic.ac.uk/~bglocker/teaching/wordlist.txt'
if not Path('wordlist.txt').exists():
    print("downloading word list...")
    urlretrieve(url, 'wordlist.txt')
    print('acquired word list.')
    
with open('wordlist.txt') as f:
    # here we use a *set* comprehension - just
    # like we've done with lists in the past but
    # the result is a set so each element is
    # guaranteed to be unique.
    # https://docs.python.org/3/tutorial/datastructures.html#sets
    # note that you can loop over a set just like you would a list
    wordlist = {l.strip() for l in f.readlines()}
    print("loaded set of words into 'wordlist' variable")

loaded set of words into 'wordlist' variable


## Task 2: Levenshtein Distance

### 2a. Implement `levenshtein_distance`

Complete the below definition for `levenshtein_distance`. Do not change the name of the function or it's arguments. 


Hints:

- You are given access to numpy (`np`). Numpy is the crown jewel of the scientific Python community - it provides a multidimensional array (`np.array()`) which can be very convenient to solve problems involving matrices.

In [2]:
# an implementation of Levenshtein Distance
def levenshtein_distance(x, y):
    # if either str has 0 length return length of the other as edit length
    xlen = len(x)
    ylen = len(y)
    if xlen == 0:
        return ylen
    if ylen == 0:
        return xlen
    
    # create dp arrays
    a = np.zeros((xlen + 1, ylen + 1))
    
    # intialise 0-th row and column
    for i in range(0, xlen + 1):
        a[i, 0] = i
    for i in range(0, ylen + 1):
        a[0, i] = i
    # computing edit length
    for i in range(1, ylen + 1):
        for j in range(1, xlen + 1):
            if y[i - 1] == x[j - 1]:
                cost = 0
            else:
                cost = 1
            
            a[j, i] = min(a[j-1, i] + 1, a[j, i-1] + 1, a[j-1, i-1] + cost)
    
    return a[xlen, ylen]

Use this test to confirm your implementation is correct.

In [3]:
print(levenshtein_distance('', 'four') == 4)
print(levenshtein_distance('smidge', '') == 6)
print(levenshtein_distance('sunny', 'snowy') == 3)
print(levenshtein_distance('algorithm', 'altruistic') == 6)
print(levenshtein_distance('imperial', 'empirical') == 3)
print(levenshtein_distance('weird', 'wired') == 2)
print(levenshtein_distance('inconsidrable', 'able') == 9)

# note this function is not written by us
# taken from https://www.stavros.io/posts/finding-the-levenshtein-distance-in-python/
# to be used as an oracle for testing only, please ignore this
def testl(first, second):
    """Find the Levenshtein distance between two strings."""
    if len(first) > len(second):
        first, second = second, first
    if len(second) == 0:
        return len(first)
    first_length = len(first) + 1
    second_length = len(second) + 1
    distance_matrix = [[0] * second_length for x in range(first_length)]
    for i in range(first_length):
       distance_matrix[i][0] = i
    for j in range(second_length):
       distance_matrix[0][j]=j
    for i in range(1, first_length):
        for j in range(1, second_length):
            deletion = distance_matrix[i-1][j] + 1
            insertion = distance_matrix[i][j-1] + 1
            substitution = distance_matrix[i-1][j-1]
            if first[i-1] != second[j-1]:
                substitution += 1
            distance_matrix[i][j] = min(insertion, deletion, substitution)
    return distance_matrix[first_length-1][second_length-1]

from random import choice, randint
from string import ascii_lowercase
# max length of string used for each test
maxl = 100
# number of test cases
testcases = 1000
# generate a random string for testing
def randomString():
    return ''.join(choice(ascii_lowercase) for i in range(randint(0, maxl)))

p = 'pass'
for i in range(testcases):
    s1 = randomString()
    s2 = randomString()
    if not(testl(s1, s2) == levenshtein_distance(s1, s2)):
        print(s1)
        print(s2)
        print(testl(s1, s2))
        print(levenshtein_distance(s1, s2))
        p = 'fail'
        break
print(p)

True
True
True
True
True
True
True
pass


### 2b. Find the minimum levenshtein distance

Use your `levenshtein_distance` function to find the `closest_match` between a `candidate` word and an iterable of `words`. Note that if multiple words from `words` share the minimal edit distance to the `candidate`, you should return the word which would come first in a dictionary. 

As a concrete example, `zark` has an edit distance of 1 with both `ark` and `bark`, but you would return `ark` as it comes lexicographically before `bark`.

Your function should return a tuple of two values - first the closest word match, and secondly the edit distance between this word and the candidate.

```python
closest, distance = closest_match('zark', ['ark', 'bark', ...])
assert closest == 'ark'
assert distance == 1
```

In [7]:
def closest_match(candidate, words):
    
    # Get arbitrary word to start with
    closest_word = words.pop()
    distance = levenshtein_distance(candidate, closest_word)
    
    for word in words:
        try_distance = levenshtein_distance(candidate, word)
        # if 2 wrods have same distance return the one with less exicographical order
        if (try_distance < distance) or ((try_distance == distance) and (word < closest_word)):
            distance = try_distance        
            closest_word = word

    return (closest_word, distance)

Run the below cell to test your implementation

In [8]:
# A one liner that queries closest_match and then prints the result
print_closest = lambda w, wl: print('{}: {} ({})'.format(w, *closest_match(w, wl)))

print_closest('zilophone', wordlist)
print_closest('inconsidrable', wordlist)
print_closest('bisamfiguatd', wordlist)

zilophone: xylophone (2.0)
inconsidrable: inconsiderable (1.0)
bisamfiguatd: disambiguate (3.0)


**Discuss in a few lines the running time of `closest_match`. Can you propose any ideas for making this faster? (Only discuss those in words, no need to do any implementations, unless you want to.)**

Let the number of words be k, length of candidate be m and average length of words be m, then the runtime of `closest_match()` is O(m*n*k), since each run of `levenshtein_distance()` is approximately O(k * m), and that `closest_match()` has to iterate through all the words to find the word with the least edit distance and lexicographical order. One method of improve running time is to edit `levenshtein_distance()` to take an extra parameter, `threshold`. Where `threshold` is the current minimum edit distance, if the current computation already has a minimum edit distance greater than `threshold`, than the computation of edit distance should abort and go to the next string. Second optimisation is to precompute a prefix tree of words and search the tree using a depth search and recording the current minimum. Once a branch has edit distance greater than current miniumu, all further sub branches should be pruned and should no longer be considered.